In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain.vectorstores import Chroma

In [44]:
loader = TextLoader('video_transcription/final_file')

In [45]:
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=0)
docs = text_splitter.split_documents(docs)


In [46]:
db = Chroma.from_documents(docs)


Using embedded DuckDB without persistence: data will be transient
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [25]:
query = "What is Batch Normalisation?"
ans = db.similarity_search(query,k=1)

In [26]:
ans = ans[0].page_content

In [41]:
from getpass import getpass

OPENAI_API_TOKEN = getpass()

In [42]:
import os
os.environ["OPENAI_API_KEY"] = OPENAI_API_TOKEN

In [43]:
from langchain import PromptTemplate, LLMChain, OpenAI
llm = OpenAI(temperature=0)
template="""Please use the following context to answer questions.
Context: {ans}
Question: {query}
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["ans","query"])
question = "What is batch normalisation?"
llm_chain = LLMChain(prompt=prompt, llm=llm)
print(llm_chain.predict(ans = ans, query = question))

 Batch normalisation is a layer that can be sprinkled throughout a deep neural network. It works by taking the activations and taking the mean and standard deviation to center the data. It also requires a gain and bias which are trainable. In order to do inference, the mean and standard deviation must be estimated.
